# Feature Extraction Continued

In [ ]:
import os

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats

import activity_classifier_utils

Load the data

In [ ]:
fs = 256
data = activity_classifier_utils.LoadWristPPGDataset()

### Features
Time Domain:
* mean
* std
* 5, 10, 15, 20, 25 percentile
* cross-correlation of all pairs of channels
* total energy

Frequency Domain:
* dominant frequency
* fraction of energy in each 1Hz bin from 0 to 6 Hz
* spectral entropy of each channel - i'll do

Low-pass filter at 12 Hz

In [ ]:
def LowpassFilter(signal, fs):
    b, a = sp.signal.butter(3, 12, btype='lowpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

Compute features

In [ ]:
def Featurize(accx, accy, accz, fs):
    """A partial featurization of the accelerometer signal.
    
    Args:
        accx: (np.array) x-channel of the accelerometer.
        accy: (np.array) y-channel of the accelerometer.
        accz: (np.array) z-channel of the accelerometer.
        fs: (number) the sampling rate of the accelerometer
        
    Returns:
        n-tuple of accelerometer features
    """
    
    accx = LowpassFilter(accx, fs)
    accy = LowpassFilter(accy, fs)
    accz = LowpassFilter(accz, fs)
    
    # The mean of the x-channel
    mn_x = np.mean(accx)

    # The standard deviation of the x-channel
    std_x = np.std(accx)

    # The 5th percentile of the x-channel
    p5_x = np.percentile(accx, 5)

    # The pearson correlation coefficient between the x and y channels
    corr_xy = sp.stats.pearsonr(accx, accy)[0]

    # The total AC energy of the x-axis
    energy_x = np.sum(np.square(accx - np.mean(accx)))  # np.var(accx) * len(accx)
    
    # Take an FFT of the signal. If the signal is too short, 0-pad it so we have at least 2046 points in the FFT.
    fft_len = max(len(accx), 2046)
    
    # Create an array of frequency bins
    freqs = np.fft.rfftfreq(fft_len, 1 / fs)
    
    # Take an FFT of the centered signal
    fft_x = np.fft.rfft(accx - np.mean(accx), fft_len)
    
    # The frequency with the most power between 0.25 and 12 Hz
    low_freqs = (freqs >= 0.25) & (freqs <= 12)
    dominant_frequency_x = freqs[low_freqs][np.argmax(np.abs(fft_x)[low_freqs])]

    # The fraction of energy between 2 and 3 Hz in the x-channel
    spectral_energy_x = np.square(np.abs(fft_x))
    energy_23_x = (np.sum(spectral_energy_x[(freqs >= 2) & (freqs <= 3)])
                   / np.sum(spectral_energy_x))
    
    return (mn_x,
            std_x,
            p5_x,
            corr_xy,
            energy_x,
            dominant_frequency_x,
            energy_23_x)

There are a lot of features, because we have to compute each of these for all channels. I've spared you that effort and put all the features in `activity_classifier_utils.py`. Poke through that file now to see the feature extraction code.

### Feature Extraction
Now we can extract the features for all of our data.

Train on 10 second long non-overlapping windows

In [ ]:
window_length_s = 10
window_shift_s = 10

In [ ]:
window_length = window_length_s * fs
window_shift = window_shift_s * fs
labels, subjects, features = [], [], []
for subject, activity, df in data:
    for i in range(0, len(df) - window_length, window_shift):
        window = df[i: i + window_length]
        accx = window.accx.values
        accy = window.accy.values
        accz = window.accz.values
        features.append(activity_classifier_utils.Featurize(accx, accy, accz, fs=fs))
        labels.append(activity)
        subjects.append(subject)

In [ ]:
labels = np.array(labels)
subjects = np.array(subjects)
features = np.array(features)

In [ ]:
labels

In [ ]:
subjects

In [ ]:
features

In [ ]:
features.shape

We started with 10 seconds of 256 Hz accelerometer data. That's 2500 samples per channel, and for three channel that's 7500 points. We've successfully reduced these 7500 points to just 55 points while hopefully retaining all the information we need to build a good classifier.

Although we only have 8 subjects of data, we have 611 datapoints because each 10 second window is its own datapoint. However, our datapoints are not independent. Because there's homogeneity in how individuals do an activity, datapoints from the same person might be more similar to each other. We have to keep this in mind when we train and evaluate our model. In the next video we'll use these features to build a random forest model and classify our data.